In [4]:
import sktime
import sys 
sys.path.append('../')
import numpy as np 
import pandas as pd 
from pickle import dump, load
# data = load(open('data/data.pkl', 'rb'))
# x = data['xvar']; y = data['yvar']
# y.name = 'target'
# print(x.shape, y.shape)


# ohlc = pd.read_csv('data/ohlcv.csv', parse_dates = ['timestamp'])


data = pd.read_csv('data/tsne_data.csv', parse_dates=['date'], index_col='date')

data

,tsne1,tsne2,Close
date,,,
2021-09-15,-32.300250,-24.670660,378.049988
2021-09-16,-36.191723,-14.198215,378.309998
2021-09-20,22.695187,-21.614645,365.700012
2021-09-21,-44.253162,-4.788871,366.149994
2021-09-23,-27.069155,-17.882883,372.980011
...,...,...,...
2024-10-28,23.477917,7.978360,495.399994
2024-10-29,16.315449,22.229230,500.160004
2024-10-30,8.466769,20.022957,496.380005


In [2]:
from arima import arima_trend

In [3]:
ac = arima_trend()

In [173]:
ts = ohlc['Open'].to_frame().set_index(ohlc['timestamp'])


models = ac.arima_model(ts, maxord=dict(p=10,d=0,q=10))


Data is not stationary
Fitting: 100 models


Fitting ARIMA models: 100%|██████████| 100/100 [33:31<00:00, 20.12s/it]


In [98]:
models['best']['results']

,name,model,mse,mae,mape,mspe,pm,r2,aic,bic,hqic,adf
"(9, 0, 10)",series,"ARIMA(9, 0, 10)",0.000242,0.011159,NaN,NaN,NaN,0.100305,-6654.58607,-6547.347341,-6614.223028,5.523200e-11


In [165]:
# autogluon.timeseries.models.AutoARIMAModel
# from autogluon.timeseries.models import AutoARIMAModel
from autogluon.timeseries import TimeSeriesDataFrame

tdf = ts.copy().drop_duplicates()
tdf = y.copy().drop_duplicates().to_frame(name = 'target')
tdf['item_id'] = 0
# tdf.index.name = 'timestamp'
# tsdf = TimeSeriesDataFrame(tdf.reset_index())
tdf

,target,item_id
date,,
2021-09-15,-0.001611,0
2021-09-16,-0.018348,0
2021-09-21,0.016872,0
2021-09-23,0.000613,0
2021-09-24,-0.010550,0
...,...,...
2025-01-10,-0.010344,0
2025-01-13,-0.004778,0
2025-01-14,0.019676,0


In [166]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA
from statsforecast.arima import arima_string

models = [AutoARIMA(max_p = 10, max_q = 10, ic = 'aic', stationary = False, nmodels=100,)]
sf = StatsForecast(models=models, freq = 'D')
sf.fit(df = tdf.reset_index(), time_col='date', id_col = 'item_id', target_col='target')
print(arima_string(sf.fitted_[0,0].model_))

ARIMA(1,0,1) with zero mean    


In [160]:
sf.fitted_[0,0].model_['aic']

-4328.178929230866

In [152]:
models = ac.arima_model(
    tdf.reset_index().drop(columns = ['item_id']).set_index('date'),
    maxord=dict(p=10,d=0,q=10)
)

Data is stationary
Fitting: 100 models


Fitting ARIMA models: 100%|██████████| 100/100 [02:10<00:00,  1.31s/it]


In [154]:
models['summary']   

,name,model,mse,mae,mape,mspe,pm,r2,aic,bic,hqic,adf
"(1, 0, 1)",series,"ARIMA(1, 0, 1)",0.000246,0.011274,NaN,NaN,NaN,0.083862,-6666.381017,-6645.954593,-6658.692819,5.734150e-11
"(2, 0, 1)",series,"ARIMA(2, 0, 1)",0.000246,0.011284,NaN,NaN,NaN,0.085030,-6665.940544,-6640.407513,-6656.330296,5.734150e-11
"(3, 0, 1)",series,"ARIMA(3, 0, 1)",0.000246,0.011248,NaN,NaN,NaN,0.085976,-6665.199726,-6634.560089,-6653.667428,5.734150e-11
"(4, 0, 1)",series,"ARIMA(4, 0, 1)",0.000246,0.011245,NaN,NaN,NaN,0.086119,-6663.390883,-6627.644640,-6649.936536,5.734150e-11
"(5, 0, 1)",series,"ARIMA(5, 0, 1)",0.000246,0.011246,NaN,NaN,NaN,0.086122,-6661.394809,-6620.541960,-6646.018413,5.734150e-11
...,...,...,...,...,...,...,...,...,...,...,...,...
"(6, 0, 10)",series,"ARIMA(6, 0, 10)",0.000245,0.011213,NaN,NaN,NaN,0.088957,-6645.177031,-6553.258120,-6610.580138,5.734150e-11
"(7, 0, 10)",series,"ARIMA(7, 0, 10)",0.000243,0.011192,NaN,NaN,NaN,0.096853,-6653.762643,-6556.737126,-6617.243700,5.734150e-11
"(8, 0, 10)",series,"ARIMA(8, 0, 10)",0.000241,0.011194,NaN,NaN,NaN,0.101970,-6658.646338,-6556.514215,-6620.205346,5.734150e-11
"(9, 0, 10)",series,"ARIMA(9, 0, 10)",0.000242,0.011186,NaN,NaN,NaN,0.099850,-6653.803624,-6546.564896,-6613.440583,5.734150e-11
